# Historical Public Debt Data

## Scopo del progetto

## Dati


- Country Name 
- Year 
- Gross Government Debt 
- Gross Domestic Product 
- Debt to GDP Ratio

In [10]:
import pandas as pd
df = pd.read_csv("HPDD_06-13-2016 19-49-43-03_timeSeries.csv")

In [46]:
def getCountries(df, countryLabel):
    if df.empty:
        print('DataFrame is empty!')
    elif countryLabel not in df.columns:
        print(countryLabel+' is not in DataFrame columns set!')
    else:
        df1 = df.drop_duplicates(subset=countryLabel, keep="last")
        return df1[countryLabel].tolist()
    return

def getIndicator(df, IndicatorLabel):
    if df.empty:
        print('DataFrame is empty!')
    elif IndicatorLabel not in df.columns:
        print(IndicatorLabel+' is not in DataFrame columns set!')
    else:
        df1 = df.drop_duplicates(subset=IndicatorLabel, keep="last")
        return df1[IndicatorLabel].tolist()
    return

def countryHasIndicator(df, countryLabel, IndicatorLabel):
    count = 0
    for x in getCountries(df, countryLabel):
        for y in getIndicator(df, IndicatorLabel):
            found = df[(df[countryLabel] == x) & (df[IndicatorLabel] == y)]
            if found.empty:
                print(x+' has no indicator: '+y)
            else:
                count += 1
    if count == len(getCountries(df, countryLabel))*len(getIndicator(df, IndicatorLabel)):
        print('Every country has all the indicators!')
    else:
        print('Not all the countries are associated with every indicators!')
                
getCountries(df, 'Country Name')
getIndicator(df, 'Indicator Name')
countryHasIndicator(df, 'Country Name', 'Indicator Name')

['Gross Domestic Product', 'Gross Government Debt', 'Debt to GDP Ratio']